9. Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.

The book's model:
It reaches about 99.2% accuracy on the test set. This places this model roughly in the top 20% in the [MNIST Kaggle competition](https://www.kaggle.com/c/digit-recognizer/) (if we ignore the models with an accuracy greater than 99.79% which were most likely trained on the test set, as explained by Chris Deotte in [this post](https://www.kaggle.com/c/digit-recognizer/discussion/61480)). Can you do better? To reach 99.5 to 99.7% accuracy on the test set, you need to add image augmentation, batch norm, use a learning schedule such as 1-cycle, and possibly create an ensemble.

In [3]:
import tensorrt
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path

2023-07-01 02:13:18.662610: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
train_raw, valid_raw, test_raw = tfds.load('mnist', as_supervised=True, split=['train[10%:]', 'train[:10%]', 'test'])

train = train_raw.shuffle(10000).batch(32).prefetch(1)
valid = valid_raw.batch(32).prefetch(1)
test = test_raw.batch(32).prefetch(1)

2023-07-01 02:13:20.563975: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-01 02:13:20.630313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-01 02:13:20.630349: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-01 02:13:20.633260: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-01 02:13:20.633297: I tensorflow/compile

In [5]:
def build_callbacks(root_dir):
    root_dir = Path() / 'data' / root_dir
    checkpoints_dir = root_dir / 'checkpoints'
    tensorboard_dir = root_dir / 'tensorboard'

    early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    checkpoints_cb = tf.keras.callbacks.ModelCheckpoint(checkpoints_dir, save_best_only=True)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_dir)
    callbacks = [early_stop_cb, checkpoints_cb, tensorboard_cb]
    print(f'Tensorboard dir: {tensorboard_dir}')
    return checkpoints_dir, callbacks

def compile_model(model_builder, root_dir):
    model = model_builder()
    checkpoints_dir, callbacks = build_callbacks(root_dir)
    # if checkpoints_dir.exists():
    #     model.load_weights(checkpoints_dir)
    optimizer = tf.keras.optimizers.Nadam()
    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizer,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
    return model, callbacks

# 98.9%
def model_01():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=5, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.SeparableConv2D(200, kernel_size=5, padding='same', strides=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

def model_02():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(64, kernel_size=5, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Conv2D(128, kernel_size=3, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.1%
def model_03():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(64, kernel_size=5, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Lambda(tf.nn.local_response_normalization),
        tf.keras.layers.Conv2D(128, kernel_size=3, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.8%
def model_04():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=7, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Conv2D(200, kernel_size=3, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.6%
def model_05():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=7, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Lambda(tf.nn.local_response_normalization),
        tf.keras.layers.Conv2D(200, kernel_size=3, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.9%
def model_06():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=7, padding='same', strides=2, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Lambda(tf.nn.local_response_normalization),
        tf.keras.layers.Conv2D(200, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(200, kernel_size=7, padding='valid', activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.5%
def model_07():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=7, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(200, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.4%
def model_08():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=7, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(200, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 96.8%
def model_09():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(200, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 96.7%
def model_10():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(100, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(200, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.8%
def model_11():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 99.1%
def model_12():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 98.9%
# Like 12 but with FCN
def model_13():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, kernel_size=14, padding='valid', activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# 99%
# Like 13 but removed Separable
def model_14():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, kernel_size=14, padding='valid', activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Like 12 but with GlobalAvgPool
# 97.5%
def model_15():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Like 12
def model_16():
    return tf.keras.Sequential([
        tf.keras.layers.Rescaling(scale=1/255, input_shape=(28, 28, 1)),
        tf.keras.layers.RandomContrast(factor=0.2),
        tf.keras.layers.RandomRotation(factor=0.1),
        tf.keras.layers.Conv2D(32, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.SeparableConv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [6]:
model, callbacks = compile_model(model_16, '20-mnist-16')

Tensorboard dir: data/20-mnist-16/tensorboard


In [7]:
hist = model.fit(
    train,
    validation_data=valid,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100


2023-07-01 02:13:22.000374: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-07-01 02:13:22.000687: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-07-01 02:13:25.335907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-01 02:13:26.767674: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-01 02:13:28.511648: I tensorflow/compiler/xla/stream

1688/1688 [==============================] - ETA: 0s - loss: 1.8180 - sparse_categorical_accuracy: 0.3248

2023-07-01 02:13:48.609687: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-07-01 02:13:48.609995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-07-01 02:13:49.818131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,28,28,1]
	 [[{{nod

INFO:tensorflow:Assets written to: data/20-mnist-16/checkpoints/assets


...

2023-07-01 02:23:09.630850: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,28,28,1]
	 [[{{node inputs}}]]
2023-07-01 02:23:09.645225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-07-01 02:23:09.652751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-07-01 02

INFO:tensorflow:Assets written to: data/20-mnist-16/checkpoints/assets


INFO:tensorflow:Assets written to: data/20-mnist-16/checkpoints/assets


1688/1688 [==============================] - 19s 11ms/step - loss: 0.3565 - sparse_categorical_accuracy: 0.8860 - val_loss: 0.1876 - val_sparse_categorical_accuracy: 0.9423
Epoch 32/100
1688/1688 [==============================] - 18s 10ms/step - loss: 0.3487 - sparse_categorical_accuracy: 0.8866 - val_loss: 0.2100 - val_sparse_categorical_accuracy: 0.9363
Epoch 33/100
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3503 - sparse_categorical_accuracy: 0.8882 - val_loss: 0.1942 - val_sparse_categorical_accuracy: 0.9405
Epoch 34/100
 546/1688 [========>.....................] - ETA: 11s - loss: 0.3349 - sparse_categorical_accuracy: 0.8912

KeyboardInterrupt: 

In [ ]:
print(model.evaluate(test))